# 1. Pima Indians Diabetes Dataset

皮马印第安人糖尿病数据集根据现有的医疗信息预测5年内皮马印第安人糖尿病发作的概率。

这是两类(2-class)分组问题，每组的观察标本量不同。共有768个观测值，8个输入变量和1个输出变量。变量名称如下：

```
0.怀孕次数。
1. 口服葡萄糖耐量试验中血浆葡萄糖浓度为2小时。
2. 舒张压（mm Hg）。
3. 三头肌组织褶厚度（mm）。
4. 2小时血清胰岛素（μU/ ml）。
5. 体重指数（kg/ （身高(m)）^ 2）。
6. 糖尿病系统功能。
7. 年龄（岁）。
8. 类变量（0或1）。
```

In [1]:
from pandas import read_csv
import numpy
dataset = read_csv('pima-indians-diabetes.data', header=None)
print(dataset.describe())

                0           1           2           3           4           5  \
count  768.000000  768.000000  768.000000  768.000000  768.000000  768.000000   
mean     3.845052  120.894531   69.105469   20.536458   79.799479   31.992578   
std      3.369578   31.972618   19.355807   15.952218  115.244002    7.884160   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      1.000000   99.000000   62.000000    0.000000    0.000000   27.300000   
50%      3.000000  117.000000   72.000000   23.000000   30.500000   32.000000   
75%      6.000000  140.250000   80.000000   32.000000  127.250000   36.600000   
max     17.000000  199.000000  122.000000   99.000000  846.000000   67.100000   

                6           7           8  
count  768.000000  768.000000  768.000000  
mean     0.471876   33.240885    0.348958  
std      0.331329   11.760232    0.476951  
min      0.078000   21.000000    0.000000  
25%      0.243750   24.000000    0.000000  
50%   

我们可以看到有最小值为零（0）的列。在某些列上，零值无效，表示为无效值或缺失值。

具体来说，以下列具有无效的零最小值：

```
1：血浆葡萄糖浓度
2：舒张压
3：三头肌组织褶厚度
4：2小时血清胰岛素
5：体重指数
```

In [2]:
print(dataset.head(20))

     0    1   2   3    4     5      6   7  8
0    6  148  72  35    0  33.6  0.627  50  1
1    1   85  66  29    0  26.6  0.351  31  0
2    8  183  64   0    0  23.3  0.672  32  1
3    1   89  66  23   94  28.1  0.167  21  0
4    0  137  40  35  168  43.1  2.288  33  1
5    5  116  74   0    0  25.6  0.201  30  0
6    3   78  50  32   88  31.0  0.248  26  1
7   10  115   0   0    0  35.3  0.134  29  0
8    2  197  70  45  543  30.5  0.158  53  1
9    8  125  96   0    0   0.0  0.232  54  1
10   4  110  92   0    0  37.6  0.191  30  0
11  10  168  74   0    0  38.0  0.537  34  1
12  10  139  80   0    0  27.1  1.441  57  0
13   1  189  60  23  846  30.1  0.398  59  1
14   5  166  72  19  175  25.8  0.587  51  1
15   7  100   0   0    0  30.0  0.484  32  1
16   0  118  84  47  230  45.8  0.551  31  1
17   7  107  74   0    0  29.6  0.254  31  1
18   1  103  30  38   83  43.3  0.183  33  0
19   1  115  70  30   96  34.6  0.529  32  1


In [3]:
print((dataset[[1,2,3,4,5]] == 0).sum())

1      5
2     35
3    227
4    374
5     11
dtype: int64


我们可以看到1,2和5列只有几个零值，而第3列和第4列显示几乎一半的行都为零值。

这充分表明，不同列可能需要不同的策略来处理，例如确保仍有足够的数据来训练预测模型。

在Python中，特别是Pandas，NumPy和Scikit-Learn，我们将缺失值标记为NaN。在sum，count等操作中，NaN值的值将被忽略。

我们可以通过使用Pandas DataFrame里的replace（）函数，在感兴趣的列的子集上 轻松地将缺失值标记为NaN 。

在我们标记了缺失值之后，我们可以使用isnull（）函数将数据集中的所有NaN值标记为真，并获取每列缺失值的计数。

In [4]:
# mark zero values as missing or NaN
dataset[[1,2,3,4,5]] = dataset[[1,2,3,4,5]].replace(0, numpy.NaN)
# count the number of NaN values in each column
print(dataset.isnull().sum())

0      0
1      5
2     35
3    227
4    374
5     11
6      0
7      0
8      0
dtype: int64


在数据集中缺少值可能会导致机器学习算法产生错误。

我们将尝试评估带有缺失值的数据对线性判别分析（LDA）算法的影响。

当数据集中缺少值时，这种算法将无效。
 
下面的例子标记了数据集中的缺失值，就像我们在上一节中所做的那样，然后尝试使用3倍交叉验证来评估LDA，求平均精度。

In [5]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

values = dataset.values
X = values[:,0:8]
y = values[:,8]

# evaluate an LDA model on the dataset using k-fold cross validation
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
print(result.mean())

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

我们无法在有缺失值的数据集上评估LDA和其他一些算法。

现在，我们来看看处理缺失值的方法。

最简单的方法就是删除包含缺失值的行。

In [6]:
# drop rows with missing values
dataset.dropna(inplace=True)
# summarize the number of rows and columns in the dataset
print(dataset.shape)

(392, 9)


In [7]:
values = dataset.values
X = values[:,0:8]
y = values[:,8]

# evaluate an LDA model on the dataset using k-fold cross validation
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
print(result.mean())

0.78582892934


另一种方法则是估算丢失值。

引用(imputing),指使用模型替换缺失值。
在替换缺少的值时可以考虑许多选择，例如：

```
• 在域内具有含义的常量值，例如0，不同于其他所有值。
• 来自另一个随机记录的值。
• 该列的平均值，中值或模式值。
• 由另一预测模型估计的值。
```

In [8]:
dataset = read_csv('pima-indians-diabetes.data', header=None)
# mark zero values as missing or NaN
dataset[[1,2,3,4,5]] = dataset[[1,2,3,4,5]].replace(0, numpy.NaN)
# fill missing values with mean column values
dataset.fillna(dataset.mean(), inplace=True)

values = dataset.values
X = values[:,0:8]
y = values[:,8]

# evaluate an LDA model on the dataset using k-fold cross validation
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
print(result.mean())

0.766927083333


当缺少数据时，并不是所有的算法都会失效。

有一些可以灵活对待缺失值的算法，例如k-Nearest Neighbors，当值缺失时，它可以将其不计入距离测量。
 
另一些算法，例如分类和回归树，可以在构建预测模型时将缺失值看作唯一且不同的值。

遗憾的是，决策树和k-Nearest Neighbors对于缺失值并不友好。

Ref   

https://mp.weixin.qq.com/s/qwXkrBO4kKh0Y5dM73aG0A   
http://machinelearningmastery.com/handle-missing-data-python/   